In this exercise, you'll apply what you learned in the **Handling missing values** tutorial.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [1]:
from learntools.core import binder
binder.bind(globals())
from learntools.data_cleaning.ex1 import *
print("Setup Complete")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Setup Complete


# 1) Take a first look at the data

Run the next code cell to load in the libraries and dataset you'll use to complete the exercise.

In [2]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
sf_permits = pd.read_csv("../input/building-permit-applications-data/Building_Permits.csv")

# set seed for reproducibility
np.random.seed(0) 

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Use the code cell below to print the first five rows of the `sf_permits` DataFrame.

In [3]:
# TODO: Your code here!


In [4]:
#%%RM_IF(PROD)%%
sf_permits.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


Does the dataset have any missing values?  Once you have an answer, run the code cell below to get credit for your work.

In [5]:
# Check your answer (Run this code cell to receive credit!)
q1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

The first five rows of the data does show that several columns have missing values.  You can see this in the "Street Number Suffix", "Proposed Construction Type" and "Site Permit" columns, among others.

In [6]:
# Line below will give you a hint
#_COMMENT_IF(PROD)_
q1.hint()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use `sf_permits.head()` to view the first five rows of the data.

# 2) How many missing data points do we have?

What percentage of the values in the dataset are missing?

In [7]:
# TODO: Your code here!
percent_missing = ____

# Check your answer
q2.check()

<IPython.core.display.Javascript object>

<span style="color:#ccaa33">Check:</span> When you've updated the starter code, `check()` will tell you whether your code is correct. You need to update the code that creates variable `percent_missing`

In [8]:
#%%RM_IF(PROD)%%
# get the number of missing data points per column
percent_missing = sf_permits.isnull().sum().sum()
q2.assert_check_failed()

<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> Incorrect value for `percent_missing`: `2245941`

In [9]:
#%%RM_IF(PROD)%%
# get the number of missing data points per column
missing_values_count = sf_permits.isnull().sum()

# how many total missing values do we have?
total_cells = np.product(sf_permits.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
q2.assert_check_passed()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

# 3) Figure out why the data is missing

Look at the columns **"Street Number Suffix"** and **"Zipcode"** from the [San Francisco Building Permits dataset](https://www.kaggle.com/aparnashastry/building-permit-applications-data). Both of these contain missing values. 
- Which, if either, are missing because they don't exist? 
- Which, if either, are missing because they weren't recorded?  

Once you have an answer, run the code cell below.

In [10]:
# Check your answer (Run this code cell to receive credit!)
q3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

If a value in the "Street Number Suffix" column is missing, it is likely because it does not exist. If a value in the "Zipcode" column is missing, it was not recorded.

In [11]:
# Line below will give you a hint
#_COMMENT_IF(PROD)_
q3.hint()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Do all addresses generally have a street number suffix?  Do all addresses generally have a zipcode?

# 4) Drop missing values: rows

If you removed all of the rows of `sf_permits` with missing values, how many rows are left?

**Note**: Do not change the value of `sf_permits` when checking this.  

In [12]:
# TODO: Your code here!


In [13]:
#%%RM_IF(PROD)%%
sf_permits.dropna()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID


Once you have an answer, run the code cell below.

In [14]:
# Check your answer (Run this code cell to receive credit!)
q4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

There are no rows remaining in the dataset!

In [15]:
# Line below will give you a hint
#_COMMENT_IF(PROD)_
q4.hint()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use `sf_permits.dropna()` to drop all missing rows.

# 5) Drop missing values: columns

Now try removing all the columns with empty values.  
- Create a new DataFrame called `sf_permits_with_na_dropped` that has all of the columns with empty values removed.  
- How many columns were removed from the original `sf_permits` DataFrame? Use this number to set the value of the `dropped_columns` variable below.

In [16]:
# TODO: Your code here
sf_permits_with_na_dropped = ____

dropped_columns = ____

# Check your answer
q5.check()

<IPython.core.display.Javascript object>

<span style="color:#ccaa33">Check:</span> When you've updated the starter code, `check()` will tell you whether your code is correct. You need to update the code that creates variables `sf_permits_with_na_dropped`, `dropped_columns`

In [22]:
#%%RM_IF(PROD)%%
# remove all columns with at least one missing value
sf_permits_with_na_dropped = sf_permits.dropna(axis=1)

# calculate number of dropped columns
cols_in_original_dataset = sf_permits.shape[1]
cols_in_na_dropped = sf_permits_with_na_dropped.shape[1]
dropped_columns = cols_in_original_dataset - cols_in_na_dropped 

q5.assert_check_passed()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

# 6) Fill in missing values automatically

Try replacing all the NaN's in the `sf_permits` data with the one that comes directly after it and then replacing any remaining NaN's with 0.  Set the result to a new DataFrame `sf_permits_with_na_imputed`.

In [18]:
# TODO: Your code here
sf_permits_with_na_imputed = ____

# Check your answer
q6.check()

<IPython.core.display.Javascript object>

<span style="color:#ccaa33">Check:</span> When you've updated the starter code, `check()` will tell you whether your code is correct. You need to update the code that creates variable `sf_permits_with_na_imputed`

In [19]:
#%%RM_IF(PROD)%%
sf_permits_with_na_imputed = sf_permits_with_na_dropped.fillna(method='bfill', axis=0).fillna(0)
q6.assert_check_failed()

<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> Expected `sf_permits_with_na_imputed` to have column `Street Number Suffix`

In [20]:
#%%RM_IF(PROD)%%
sf_permits_with_na_imputed = sf_permits.fillna(method='bfill', axis=0).fillna(0)
q6.assert_check_passed()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

# More practice

If you're looking for more practice handling missing values:

* Check out [this noteboook](https://www.kaggle.com/alexisbcook/missing-values) on handling missing values using scikit-learn's imputer. 
* Look back at the "Zipcode" column in the `sf_permits` dataset, which has some missing values. How would you go about figuring out what the actual zipcode of each address should be? (You might try using another dataset. You can search for datasets about San Fransisco on the [Datasets listing](https://www.kaggle.com/datasets).) 

# Keep going

In the next lesson, learn how to [**apply scaling and normalization**](#$NEXT_NOTEBOOK_URL$) to transform your data.